<div style = 'text-align: center;'>
    <img src = '../images/ga_logo_large.png'>
</div>

# **Project 2: Ames Price Prediction Model**

---
### **Problem Statement**
In the United States people value spacious properties. Therefore, it is assumed that the more living space is available in a house, and the bigger the lot, the higher the price the property would command in the local market. This analysis aims to build a predictive pricing model based on that premise.<br>
The model will utilize multiple linear regression techniques, and the model's performance will be assessed using the metric Root Mean Squared Error (RMSE).

---
### **Data**
Only one dataset is used for this analysis and predictive model.<br>
* `train.csv`: This dataset is derived from the original source named `AmesHousing.txt`.  It contains information obtained from the Ames Assessor's Office, which is locally used to compute assessment values for residential properties sold in Ames, IA between 2006 and 2010.

---
### **Consulted Sources**
This analysis also relies on the outiside sources listed below.  These were consulted to gain a better understanding of what drives home sale prices across all markets in the U.S.<br>
* <a href =  'https://www.opendoor.com/articles/factors-that-influence-home-value'>8 critical factors that influence a home's value.</a>
    * Neighborhood comps and location are very important.
    * Livable space: Home values are estimated by price per square foot.
    * Newer homes appraise at higher values.
* <a href = 'https://www.constructionhq.world/issue-sections/product-focus/exterior-walls/what-is-masonry-veneer/#:~:text=Masonry%20veneer%20is%20a%20construction,the%20masonry%20veneer%20and%20mortar.'>What is Masonry Veneer?</a>

---
### **Functions**
For the sake of efficiency and accuracy, this section houses all non-native functions applicable to this analysis.

**Create dictionary to format column names**

In [7]:
def fix_col_names(col_list):
    '''
    This function takes as argument a list of column names from a data frame.
    It returns a dictionary where each key is the current column name and the 
    corresponding value is the very same column name in snake case.
    
    Parameters:
    col_list: list of strings to pass in, these are the dataframe's column names.
    
    Return:
    dictionary: organized dictionary with all new column names to pass into data 
    frame and facilitate column conversion.
    '''
    columns_dict = {}
    
    for col in col_list:
        value = col.lower().replace(' ','_')
        columns_dict[col] = value
    
    return columns_dict

**Quickly check the percent of missing values in a column**

In [9]:
def nulls_pct(df, col_label):
    '''
    This function quickly computes and prints out the percent of null values in a columm.
    
    Parameters:
    df: actual dataframe, just the name.
    col_label: column to check into in string format.
    
    Return:
    It actually prints out percent of null values right underneath the cell where function
    is called.
    '''
    count = df[df[col_label].isnull()].shape[0]
    pct = (count / df.shape[0])
    
    print(f'Missing percentage: {round(pct * 100, 2)}; Rows: {count}')

---
### Data Import & Cleaning

In [13]:
# needed libraries for this notebook

import numpy as np
import pandas as pd

In [14]:
# read in file
file_path = '../datasets/train.csv'
ames = pd. read_csv(file_path)

# check size
ames.shape

(2051, 81)

In [15]:
# check some rows
ames.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


Given the number of columns, it is best to check them using `df.info()` below.

In [17]:
# check all column nulls and data types
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

Before doing any cleanup of this data, it's best to change all column names to `snake case` format.  Not having spaces makes it easier to pick and navigate.<br>
As there are 81 columns, the `fix_col_names()` function will be used to assist with the name changes.

In [19]:
# get column names
columns = fix_col_names(ames.columns.tolist())
# columns << checked dictionary output, it looks good.  Suppressed because of length.

In [20]:
# change column names
ames.rename(columns = columns, inplace = True)

In [21]:
# check nulls and data types again, as well as column names
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2051 non-null   int64  
 1   pid              2051 non-null   int64  
 2   ms_subclass      2051 non-null   int64  
 3   ms_zoning        2051 non-null   object 
 4   lot_frontage     1721 non-null   float64
 5   lot_area         2051 non-null   int64  
 6   street           2051 non-null   object 
 7   alley            140 non-null    object 
 8   lot_shape        2051 non-null   object 
 9   land_contour     2051 non-null   object 
 10  utilities        2051 non-null   object 
 11  lot_config       2051 non-null   object 
 12  land_slope       2051 non-null   object 
 13  neighborhood     2051 non-null   object 
 14  condition_1      2051 non-null   object 
 15  condition_2      2051 non-null   object 
 16  bldg_type        2051 non-null   object 
 17  house_style   

All column names are now correctly formatted in `snake case`.  Next check the missing values for columns as needed. Then, if the data type needs to be modified, do so immediately after addressing missing values.

---
**Missing Values**<br>
Column: `lot_frontage`<br>
Continuous data column, so its data type is correct.

In [24]:
# check missing values, are they all NaN?
ames[ames['lot_frontage'].isnull()]['lot_frontage'].unique()

array([nan])

All 330 missing values in this column are NaN.

In [26]:
# statistics about this column
nulls_pct(ames, 'lot_frontage')
ames['lot_frontage'].describe()

Missing percentage: 16.09; Rows: 330


count    1721.000000
mean       69.055200
std        23.260653
min        21.000000
25%        58.000000
50%        68.000000
75%        80.000000
max       313.000000
Name: lot_frontage, dtype: float64

The mean and the median for the available values are very close to each other.  For the first iteration of this analysis and predictive model, let's fill the missing values with the mean value.

In [28]:
# replace NaNs with mean value (69.06)
indexes = ames[ames['lot_frontage'].isnull()].index.tolist()
ames.loc[indexes, 'lot_frontage'] = 69.06

In [29]:
# confirm change
nulls_pct(ames, 'lot_frontage')
ames['lot_frontage'].describe()

Missing percentage: 0.0; Rows: 0


count    2051.000000
mean       69.055973
std        21.306357
min        21.000000
25%        60.000000
50%        69.060000
75%        78.000000
max       313.000000
Name: lot_frontage, dtype: float64

All NaN values successfully replaced.

---
Column: `alley`<br>
Nonimal data.

In [32]:
# check value counts
nulls_pct(ames, 'alley')
ames['alley'].value_counts(dropna = False)

Missing percentage: 93.17; Rows: 1911


alley
NaN     1911
Grvl      85
Pave      55
Name: count, dtype: int64

Per data dictionary, this is an nominal column with three possibilities: `Grvl` for Gravel, `Pave` for Paved or `NA` for No alley access. It is safe to assume that `NA` is what's driving all the `NaN` values.<br>
All these rows need to be changed to `No_Alley`.

In [34]:
# make the change
indexes = ames[ames['alley'].isnull()].index.tolist()
ames.loc[indexes, 'alley'] = 'No_Alley'

# confirm change
nulls_pct(ames, 'alley')
ames['alley'].value_counts()

Missing percentage: 0.0; Rows: 0


alley
No_Alley    1911
Grvl          85
Pave          55
Name: count, dtype: int64

All `NaN` rows successfully addressed.

---
Column: `mas_vnr_type`<br>
Nonimal data.

In [37]:
# check value counts
nulls_pct(ames, 'mas_vnr_type')
ames['mas_vnr_type'].value_counts(dropna = False)

Missing percentage: 60.46; Rows: 1240


mas_vnr_type
NaN        1240
BrkFace     630
Stone       168
BrkCmn       13
Name: count, dtype: int64

Per data dictionary, this is a nominal column with five possible values: `BrkCmn` for Brick Common, `BrkFace` for Brick Face, `CBlock` for Cinder Block, `None` for None and `Stone` for Stone.  This is another case where it might be safe to assume that all values for `None` are driving `nulls`.<br>
These rows should be changed to `NoMsn` (no masonry).

In [39]:
# make the change
indexes = ames[ames['mas_vnr_type'].isnull()].index.tolist()
ames.loc[indexes, 'mas_vnr_type'] = 'NoMsn'

In [40]:
# confirm change
nulls_pct(ames, 'mas_vnr_type')
ames['mas_vnr_type'].value_counts()

Missing percentage: 0.0; Rows: 0


mas_vnr_type
NoMsn      1240
BrkFace     630
Stone       168
BrkCmn       13
Name: count, dtype: int64

All `NaN` rows successfully addressed.

---
Column: `mas_vnr_area`<br>
Continuous data.

In [43]:
# check missing values, all NaN?
ames[ames['mas_vnr_area'].isnull()]['mas_vnr_area'].unique()

array([nan])

All 22 values are `NaN`.

In [45]:
# column statistics
nulls_pct(ames, 'mas_vnr_area')
ames['mas_vnr_area'].describe()

Missing percentage: 1.07; Rows: 22


count    2029.000000
mean       99.695909
std       174.963129
min         0.000000
25%         0.000000
50%         0.000000
75%       161.000000
max      1600.000000
Name: mas_vnr_area, dtype: float64

Most rows have a value of 0.00 for this column.  Since there are just 22 rows with missing values, check the value of those rows under the `mas_vnr_type` column.  If the value is `NoMsn` then convert this row's value to 0.00.

In [47]:
# check against 'mas_vnr_type' column
ames[ames['mas_vnr_area'].isnull()]['mas_vnr_type'].value_counts()

mas_vnr_type
NoMsn    22
Name: count, dtype: int64

In [48]:
# change all missing values on 'mas_vnr_area' to 0
indexes = ames[ames['mas_vnr_area'].isnull()].index.tolist()
ames.loc[indexes, 'mas_vnr_area'] = 0.0

In [49]:
# confirm change
nulls_pct(ames, 'mas_vnr_area')
ames['mas_vnr_area'].describe()

Missing percentage: 0.0; Rows: 0


count    2051.000000
mean       98.626524
std       174.324690
min         0.000000
25%         0.000000
50%         0.000000
75%       159.000000
max      1600.000000
Name: mas_vnr_area, dtype: float64

All missing values successfully addressed.

---
Column: `bsmt_qual`<br>
Ordinal data.

In [52]:
# check value counts
nulls_pct(ames, 'bsmt_qual')
ames['bsmt_qual'].value_counts(dropna = False)

Missing percentage: 2.68; Rows: 55


bsmt_qual
TA     887
Gd     864
Ex     184
Fa      60
NaN     55
Po       1
Name: count, dtype: int64

Per data dictionary this is ordinal data, where the five possible values are `Ex` for Excellent, `Gd` for Good, `TA` for Typical, `Fa` for Fair, `Po` for Poor and `NA` for No Basement.  Based on these ordinal categories, it may be safe to assume that the No Basement category is what's driving the `NaN` values.<br>
Change all 55 `NaN` values on this column to '`Nb`.

In [54]:
# make the change
indexes = ames[ames['bsmt_qual'].isnull()].index.tolist()
ames.loc[indexes, 'bsmt_qual'] = 'Nb'

In [55]:
# confirm change
nulls_pct(ames, 'bsmt_qual')
ames['bsmt_qual'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmt_qual
TA    887
Gd    864
Ex    184
Fa     60
Nb     55
Po      1
Name: count, dtype: int64

All `NaN` values successfully changed to `Nb`

---
Column: `bsmt_cond` <br>
Ordinal data.

In [58]:
# check value counts
nulls_pct(ames, 'bsmt_cond')
ames['bsmt_cond'].value_counts(dropna = False)

Missing percentage: 2.68; Rows: 55


bsmt_cond
TA     1834
Gd       89
Fa       65
NaN      55
Po        5
Ex        3
Name: count, dtype: int64

This is exactly the same situation as the `bsmt_qual` column.  Take same approach and change all `NaN` values to `Nb`.

In [60]:
# make the change
indexes = ames[ames['bsmt_cond'].isnull()].index.tolist()
ames.loc[indexes, 'bsmt_cond'] = 'Nb'

In [61]:
# confirm change
nulls_pct(ames, 'bsmt_cond')
ames['bsmt_cond'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmt_cond
TA    1834
Gd      89
Fa      65
Nb      55
Po       5
Ex       3
Name: count, dtype: int64

All `Nan` values successfully changed to `Nb`.

---
Column: `bsmt_exposure` <br>
Ordinal data.

In [64]:
# check value counts
nulls_pct(ames, 'bsmt_exposure')
ames['bsmt_exposure'].value_counts(dropna = False)

Missing percentage: 2.83; Rows: 58


bsmt_exposure
No     1339
Av      288
Gd      203
Mn      163
NaN      58
Name: count, dtype: int64

Per data dictionary this column has ordinal data, where the the five possible values are `Gd` for Good Exposure, `Av` for Average Exposure, `Mn` for Minimum Exposure, `No` for No Exposure and `NA` for No Basement.  Per the observed pattern in previous columns with similar data, it is safely assumed that the `NA` categories are driving the `NaN`.<br>
Change all rows with `NaN` values to `Nb` (No Basemement).

In [66]:
# make the change
indexes = ames[ames['bsmt_exposure'].isnull()].index.tolist()
ames.loc[indexes, 'bsmt_exposure'] = 'Nb'

In [67]:
# confirm change
nulls_pct(ames, 'bsmt_exposure')
ames['bsmt_exposure'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmt_exposure
No    1339
Av     288
Gd     203
Mn     163
Nb      58
Name: count, dtype: int64

All 58 `NaN` values were successfully changed to `Nb`.

---
Column: `bsmtfin_type_1`<br>
Ordinal data.

In [70]:
# check value counts
nulls_pct(ames, 'bsmtfin_type_1')
ames['bsmtfin_type_1'].value_counts(dropna =  False)

Missing percentage: 2.68; Rows: 55


bsmtfin_type_1
GLQ    615
Unf    603
ALQ    293
BLQ    200
Rec    183
LwQ    102
NaN     55
Name: count, dtype: int64

Per the data dictionary, this column is made up of ordinal data.  It takes on seven possible values, where `GLQ` is for Good Living Quarters, `ALQ` is for Average Living Quarters, `BLQ` is for Below Average Living Quarters, `Rec` is for Average Rec Room, `LwQ` is for Low Quality, `Unf` is for Unfinished and `NA` is for No Basement.<br>
As previous similar examples, it is safely assumed that the 55 `NaN` values are driven by the `NA` category.  Change all 55 values to `Nb`.

In [72]:
# make the change
indexes = ames[ames['bsmtfin_type_1'].isnull()].index.tolist()
ames.loc[indexes, 'bsmtfin_type_1'] = 'Nb'

In [73]:
# confirm change
nulls_pct(ames, 'bsmtfin_type_1')
ames['bsmtfin_type_1'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmtfin_type_1
GLQ    615
Unf    603
ALQ    293
BLQ    200
Rec    183
LwQ    102
Nb      55
Name: count, dtype: int64

All `NaN` values successfully changed to `Nb`.

---
Column: `bsmtfin_sf_1`<br>
Continuous data.

In [76]:
# column statistics
nulls_pct(ames, 'bsmtfin_sf_1')
ames['bsmtfin_sf_1'].describe()

Missing percentage: 0.05; Rows: 1


count    2050.000000
mean      442.300488
std       461.204124
min         0.000000
25%         0.000000
50%       368.000000
75%       733.750000
max      5644.000000
Name: bsmtfin_sf_1, dtype: float64

Only one row is missing a value. Check row to see if this property has a basement or not.

In [78]:
# inspect row
ames[ames['bsmtfin_sf_1'].isnull()]['bsmt_qual']

1327    Nb
Name: bsmt_qual, dtype: object

This property has no basement.  Change missing value to 0.0.  Row index is 1327.

In [80]:
# make change
ames.loc[1327, 'bsmtfin_sf_1'] = 0.0

In [81]:
# confirm change
nulls_pct(ames, 'bsmtfin_sf_1')
ames['bsmtfin_sf_1'].describe()

Missing percentage: 0.0; Rows: 0


count    2051.000000
mean      442.084837
std       461.195041
min         0.000000
25%         0.000000
50%       368.000000
75%       733.500000
max      5644.000000
Name: bsmtfin_sf_1, dtype: float64

Missing value successfully changed to 0.0

---
Column: `bsmtfin_type_2`<br>
Ordinal data.

In [84]:
# check value counts
nulls_pct(ames, 'bsmtfin_type_2')
ames['bsmtfin_type_2'].value_counts(dropna = False)

Missing percentage: 2.73; Rows: 56


bsmtfin_type_2
Unf    1749
Rec      80
LwQ      60
NaN      56
BLQ      48
ALQ      35
GLQ      23
Name: count, dtype: int64

Per data dictionary, this column takes on the same seven ordinal values as column `bsmtfin_type_1`.  Follow the same approach and change all `NaN` values to `Nb`.

In [86]:
# make the change
indexes = ames[ames['bsmtfin_type_2'].isnull()].index.tolist()
ames.loc[indexes, 'bsmtfin_type_2'] = 'Nb'

In [87]:
# confirm change
nulls_pct(ames, 'bsmtfin_type_2')
ames['bsmtfin_type_2'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmtfin_type_2
Unf    1749
Rec      80
LwQ      60
Nb       56
BLQ      48
ALQ      35
GLQ      23
Name: count, dtype: int64

All `NaN` rows successfully changed to `Nb`.

---
Column: `bsmtfin_sf_2`<br>
Continuous data.

In [90]:
# check row index 1327, it was nan for bsmtfin_sf_1
nulls_pct(ames, 'bsmtfin_sf_2')
ames.loc[1327,'bsmtfin_sf_2']

Missing percentage: 0.05; Rows: 1


nan

In [91]:
# as suspected, change to 0.0, the property on that row has no basement
ames.loc[1327, 'bsmtfin_sf_2'] = 0.0

In [92]:
# confirm change
nulls_pct(ames, 'bsmtfin_sf_2')
ames['bsmtfin_sf_2'].describe()

Missing percentage: 0.0; Rows: 0


count    2051.000000
mean       47.935641
std       164.964052
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      1474.000000
Name: bsmtfin_sf_2, dtype: float64

`NaN` value successfully changed to 0.0

---
Columns: `bsmt_unf_sf` and `total_bsmt_sf`.<br>
Continuous data.<br>
From intial run of `ames.info()` several cells back, it was noted that these two columns have just one missing value. Check row index 1327 for both columns as this property has no basement.  If `NaN`, then change to 0.0.

In [95]:
ames.loc[1327,['bsmt_unf_sf', 'total_bsmt_sf']]

bsmt_unf_sf      NaN
total_bsmt_sf    NaN
Name: 1327, dtype: object

In [96]:
# make the changes
ames.loc[1327,['bsmt_unf_sf', 'total_bsmt_sf']] = 0.0

In [97]:
# confirm the changes
nulls_pct(ames, 'bsmt_unf_sf')
nulls_pct(ames, 'total_bsmt_sf')
ames.loc[1327,['bsmt_unf_sf', 'total_bsmt_sf']]

Missing percentage: 0.0; Rows: 0
Missing percentage: 0.0; Rows: 0


bsmt_unf_sf      0.0
total_bsmt_sf    0.0
Name: 1327, dtype: object

Both `NaN` values successfully changed to 0.0

---
Column: `bsmt_full_bath`<br>
Discrete data.

In [100]:
# check value counts
nulls_pct(ames, 'bsmt_full_bath')
ames['bsmt_full_bath'].value_counts(dropna = False)

Missing percentage: 0.1; Rows: 2


bsmt_full_bath
0.0    1200
1.0     824
2.0      23
NaN       2
3.0       2
Name: count, dtype: int64

In [101]:
# check against col 'bsmt_qual'
ames[ames['bsmt_full_bath'].isnull()]['bsmt_qual']

616     Nb
1327    Nb
Name: bsmt_qual, dtype: object

These two properties have no basement, therefore change `bsmt_full_bath` values to 0.0

In [103]:
# make the changes
indexes = ames[ames['bsmt_full_bath'].isnull()]['bsmt_qual'].index.tolist()
ames.loc[indexes, 'bsmt_full_bath'] = 0.0

In [104]:
# confirm change
nulls_pct(ames, 'bsmt_full_bath')
ames['bsmt_full_bath'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmt_full_bath
0.0    1202
1.0     824
2.0      23
3.0       2
Name: count, dtype: int64

Column successfully cleaned.

---
Column: `bsmt_half_bath`<br>
Discrete data.<br>
Check row indexes 616 and 1327 as these two properties have no basement.  If `NaN`, then change to 0.0

In [107]:
# check two rows, same indexes as previous cell
ames.loc[indexes, 'bsmt_half_bath']

616    NaN
1327   NaN
Name: bsmt_half_bath, dtype: float64

In [108]:
# make the changes
ames.loc[indexes, 'bsmt_half_bath'] = 0.0

# confirm changes
nulls_pct(ames, 'bsmt_half_bath')
ames['bsmt_half_bath'].value_counts()

Missing percentage: 0.0; Rows: 0


bsmt_half_bath
0.0    1925
1.0     122
2.0       4
Name: count, dtype: int64

Column successfully cleaned.

---
Column: `fireplace_qu`<br>
Ordinal data.

In [111]:
# check value counts
nulls_pct(ames, 'fireplace_qu')
ames['fireplace_qu'].value_counts(dropna = False)

Missing percentage: 48.76; Rows: 1000


fireplace_qu
NaN    1000
Gd      523
TA      407
Fa       59
Po       31
Ex       31
Name: count, dtype: int64

Per the data dictionary, this ordinal column takes on six different values, where `Ex` is for Excellent..., `Gd` is for Good..., `TA` is for Average..., `Fa` is for Fair..., `Po` is for Poor... and `NA` is for No Fireplace.<br>
Based on the patterns observed thus far, it is safe to assume that the `NA` label is what's likely driving all the `NaN` values.  Change them to `Nf`.

In [113]:
# make the change
indexes = ames[ames['fireplace_qu'].isnull()].index.tolist()
ames.loc[indexes,'fireplace_qu'] = 'Nf'

In [114]:
# confirm changes
nulls_pct(ames, 'fireplace_qu')
ames['fireplace_qu'].value_counts()

Missing percentage: 0.0; Rows: 0


fireplace_qu
Nf    1000
Gd     523
TA     407
Fa      59
Po      31
Ex      31
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `garage_type`<br>
Nominal data.

In [117]:
# check value counts
nulls_pct(ames, 'garage_type')
ames['garage_type'].value_counts(dropna = False)

Missing percentage: 5.51; Rows: 113


garage_type
Attchd     1213
Detchd      536
BuiltIn     132
NaN         113
Basment      27
2Types       19
CarPort      11
Name: count, dtype: int64

Per the data dictionary, this nominal column can take on seven different categories, where `2Types` is for properties with more than one type of garage, `Attchd` is for Attached to home, `Basment` is for Basement Garage, `BuiltIn` is for for the garage being part of the house, `CarPort` is for Car Ports, `Detchd` is for Detached from home and `NA` is for No Garage.<br>
The pattern repeats, it is safe to assume that the `NA` label is likely driving the `NaN` values, therefore change all these rows to `Ng`.

In [119]:
# make the changes
indexes = ames[ames['garage_type'].isnull()].index.tolist()
ames.loc[indexes, 'garage_type'] = 'Ng'

In [120]:
# confirm changes
nulls_pct(ames, 'garage_type')
ames['garage_type'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_type
Attchd     1213
Detchd      536
BuiltIn     132
Ng          113
Basment      27
2Types       19
CarPort      11
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `garage_yr_blt` and `garage_finish`<br>
Discrete and ordinal data respectively.

In [123]:
# check value counts for garage_yr_blt
nulls_pct(ames, 'garage_yr_blt')
ames['garage_yr_blt'].value_counts(dropna = False)

Missing percentage: 5.56; Rows: 114


garage_yr_blt
NaN       114
2005.0    105
2007.0     84
2006.0     80
2004.0     72
         ... 
1918.0      1
2207.0      1
1933.0      1
1895.0      1
1919.0      1
Name: count, Length: 101, dtype: int64

In [227]:
# check all unique year values
print(sorted(ames['garage_yr_blt'].unique()))

[1900.0, 1916.0, 1920.0, 1924.0, 1930.0, 1931.0, 1935.0, 1937.0, 1939.0, 1949.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1961.0, 1963.0, 1964.0, 1965.0, 1966.0, 1968.0, 1969.0, 1971.0, 1972.0, 1973.0, 1976.0, 1977.0, 1978.0, 1979.0, 1981.0, 1986.0, 1992.0, 1994.0, 1995.0, 1996.0, 1997.0, 1999.0, 2000.0, 2001.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2009.0, nan, 1895.0, 1896.0, 1910.0, 1914.0, 1915.0, 1917.0, 1918.0, 1919.0, 1921.0, 1922.0, 1923.0, 1925.0, 1926.0, 1927.0, 1928.0, 1929.0, 1932.0, 1933.0, 1934.0, 1936.0, 1938.0, 1940.0, 1941.0, 1942.0, 1945.0, 1946.0, 1947.0, 1948.0, 1950.0, 1962.0, 1967.0, 1970.0, 1974.0, 1975.0, 1980.0, 1982.0, 1983.0, 1984.0, 1985.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1993.0, 1998.0, 2002.0, 2008.0, 2010.0, 2207.0]


These years are listed as floats, convert to integers.  Also, look into rows for which the year is **2207**.

In [124]:
# check value counts for garage_finish
nulls_pct(ames, 'garage_finish')
ames['garage_finish'].value_counts(dropna = False

Missing percentage: 5.56; Rows: 114


garage_finish
Unf    849
RFn    579
Fin    509
NaN    114
Name: count, dtype: int64

Both columns have the same number of missing values.  But, since `garage_finsh` takes on four distinct ordinal values, it makes sense to fix this column first, because it may help with fixing `garage_yr_blt`.<br>
The possible categories for `garage_finish` are `Fin` for Finished, `RFn` for Rough Finished, `Unf` for Unfinished and `NA` for No Garage.  Per the observed pattern on this dataset, it is safe to assume that all `NaN` values in the `garage_finish` column are likely driven by the label `NA`.  Change these values to `Ng` (No Garage).

In [126]:
# make changes
indexes = ames[ames['garage_finish'].isnull()].index.tolist()
ames.loc[indexes, 'garage_finish'] = 'Ng'

In [127]:
# confirm changes
nulls_pct(ames, 'garage_finish')
ames['garage_finish'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_finish
Unf    849
RFn    579
Fin    509
Ng     114
Name: count, dtype: int64

All values successfully changed to `Ng`.  Now compare the current indexes with the indexes for `NaN` in the `garage_yr_blt` column to see if they are the same 114 rows.

In [129]:
# compare indexes
ames[ames['garage_yr_blt'].isnull()].index.tolist() == indexes # the indexes variable has the most recent indexes from 'garage_finish'

True

The rows match.  However, this column stores year values in four-digit format, since these rows are for properties that have no garages, change all `NaN` cells to 9999 as integer.

In [236]:
# make changes
ames.loc[indexes, 'garage_yr_blt'] = 9999

In [238]:
# confirm changes
nulls_pct(ames, 'garage_yr_blt')
ames['garage_yr_blt'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_yr_blt
9999.0    114
2005.0    105
2007.0     84
2006.0     80
2004.0     72
         ... 
1918.0      1
2207.0      1
1933.0      1
1895.0      1
1919.0      1
Name: count, Length: 101, dtype: int64

All rows are successfully changed to 9999.  Now convert all year values to integers.

In [249]:
ames['garage_yr_blt'] = ames['garage_yr_blt'].astype(int)
# confirm change
ames['garage_yr_blt'].sort_values(ascending = False)

722     9999
114     9999
120     9999
457     9999
1814    9999
        ... 
12      1900
217     1900
520     1900
410     1896
1793    1895
Name: garage_yr_blt, Length: 2051, dtype: int32

All years are now integers.  Next, investigate row(s) where the year is **2207**.

In [252]:
ames[ames['garage_yr_blt'] == 2207]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
1699,2261,916384070,20,RL,68.0,8298,Pave,No_Alley,IR1,HLS,...,0,0,NaN,NaN,NaN,0,9,2007,New,267300


It's only one row.  Check against `year_built` column (original construction date) and assume the garage was also part of the original construction.  Assign that year value to `garage_yr_blt`.

In [257]:
ames[ames['garage_yr_blt'] == 2207]['year_built']

1699    2006
Name: year_built, dtype: int64

In [263]:
# assign 2006 to 'garage_yr_blt' column, row index 1699, and confirm
ames.loc[1699,'garage_yr_blt'] = 2006
ames.loc[1699,'garage_yr_blt']

2006

In [265]:
# print unique years again, just to make sure
print(sorted(ames['garage_yr_blt'].unique()))

[1895, 1896, 1900, 1910, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 9999]


All rows successfully cleaned.

---
Columns: `garage_qual` and `garage_cond`<br>
Both ordinal data.<br>

In [279]:
# check value counts for 'garage_qual'
nulls_pct(ames,'garage_qual')
ames['garage_qual'].value_counts(dropna = False)

Missing percentage: 5.56; Rows: 114


garage_qual
TA     1832
NaN     114
Fa       82
Gd       18
Ex        3
Po        2
Name: count, dtype: int64

In [281]:
# check value counts for 'garage_cond'
nulls_pct(ames,'garage_cond')
ames['garage_cond'].value_counts(dropna = False)

Missing percentage: 5.56; Rows: 114


garage_cond
TA     1868
NaN     114
Fa       47
Gd       12
Po        8
Ex        2
Name: count, dtype: int64

These two columns have the same number of missing values (114) as `garage_yr_blt` and `garage_finish`.  Since these two columns also deal with garage features, and one of the categories for both is `NA` (No Garage), it is once again assumed that this is what is driving the `NaN` values.  Change all `NA` rows to `Ng`.

In [296]:
# make the changes
indexes = ames['garage_qual'].isnull().index.tolist()
ames.loc[indexes,'garage_qual'] = 'Ng'

ames.loc[indexes,'garage_cond'].isnull().index.tolist() == indexes # if TRUE, use same indexes

True

In [298]:
# make change
ames.loc[indexes, 'garage_cond'] = 'Ng'

In [302]:
# confirm changes
nulls_pct(ames, 'garage_qual')
ames['garage_qual'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_qual
Ng    2051
Name: count, dtype: int64

In [304]:
# confirm changes
nulls_pct(ames, 'garage_cond')
ames['garage_cond'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_cond
Ng    2051
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `garage_cars`<br>
Discrete data.

In [308]:
# check value counts
nulls_pct(ames,'garage_cars')
ames['garage_cars'].value_counts(dropna = False)

Missing percentage: 0.05; Rows: 1


garage_cars
2.0    1136
1.0     524
3.0     263
0.0     113
4.0      13
5.0       1
NaN       1
Name: count, dtype: int64

In [318]:
# check unique missing value against 'garage_type' col
ames[ames['garage_cars'].isnull()]['garage_type']

1712    Detchd
Name: garage_type, dtype: object

This property has a detached garage. From previous cells, it was noted the number of properties with datached garages is 536.  Check those rows to see the most prevalent value for `garage_cars`, assign that value to row index 1712. 

In [327]:
ames[ames['garage_type'] == 'Detchd']['garage_cars'].value_counts(dropna = False)

garage_cars
2.0    273
1.0    251
3.0      8
4.0      2
5.0      1
NaN      1
Name: count, dtype: int64

The most prevalent value is 2, assign that to row index 1712.

In [331]:
ames.loc[1712, 'garage_cars'] = 2

In [333]:
# confirm change
nulls_pct(ames, 'garage_cars')
ames['garage_cars'].value_counts()

Missing percentage: 0.0; Rows: 0


garage_cars
2.0    1137
1.0     524
3.0     263
0.0     113
4.0      13
5.0       1
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `garage_area`<br>
Continuous data.

In [337]:
# check column stats
nulls_pct(ames, 'garage_area')
ames['garage_area'].describe()

Missing percentage: 0.05; Rows: 1


count    2050.000000
mean      473.671707
std       215.934561
min         0.000000
25%       319.000000
50%       480.000000
75%       576.000000
max      1418.000000
Name: garage_area, dtype: float64

In [339]:
# select row with missing value, check if property has a garage
ames[ames['garage_area'].isnull()]['garage_type']

1712    Detchd
Name: garage_type, dtype: object

This is the same row from the previous cell, the `garage_cars` column.  A value of 2 was assigned to it. Check the average area of detached garages with 2-car capacity and assign that value to this row, index 1712.

In [391]:
# get mean area of detached garages with 2 car capacity

boolean_mask = (ames['garage_type'] == 'Detchd') & (ames['garage_cars'] == 2)  # << consulted chatgpt to recall this one
avg_area = ames.loc[boolean_mask, 'garage_area'].mean()
avg_area

532.7619047619048

In [394]:
# make the change
ames.loc[1712, 'garage_area'] = avg_area

In [396]:
# confirm change
nulls_pct(ames, 'garage_area')
ames['garage_area'].describe()

Missing percentage: 0.0; Rows: 0


count    2051.000000
mean      473.700518
std       215.885830
min         0.000000
25%       319.000000
50%       480.000000
75%       576.000000
max      1418.000000
Name: garage_area, dtype: float64

All rows successfully cleaned.

---
Column: `pool_qc`<br>
Ordinal data.

In [400]:
# check value counts 
nulls_pct(ames, 'pool_qc')
ames['pool_qc'].value_counts(dropna = False)

Missing percentage: 99.56; Rows: 2042


pool_qc
NaN    2042
Gd        4
Fa        2
TA        2
Ex        1
Name: count, dtype: int64

Per the data dictionary, this column can take on five different categories.  They are: `Ex` for Excellent, `Gd` for Good, `TA` for Average/Typical, `Fa` for Fair and `NA` for No Pool.<br>
Given the well observed pattern with this data set, it is safely assumed that all `NaN` values are being driven by the `NA` label.  Change all these rows to `Np`.

In [405]:
# make the changes
indexes = ames[ames['pool_qc'].isnull()].index.tolist()
ames.loc[indexes, 'pool_qc'] = 'Np'

In [407]:
# confirm changes
nulls_pct(ames, 'pool_qc')
ames['pool_qc'].value_counts()

Missing percentage: 0.0; Rows: 0


pool_qc
Np    2042
Gd       4
Fa       2
TA       2
Ex       1
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `fence`<br>
Ordinal data.

In [413]:
# check value counts
nulls_pct(ames, 'fence')
ames['fence'].value_counts(dropna = False)

Missing percentage: 80.5; Rows: 1651


fence
NaN      1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: count, dtype: int64

Per data dictionary this column can take on five different values.  They are: `GdPrv` for Good Privacy, `MnPrv` for Minimum Privacy, `GdWo` for Good Wood, `MnWw` for Minimum Wood/Wire and `NA` for No Fence.<br>
Change all `NA` values to `NoFe`.

In [420]:
# make the changes
indexes = ames[ames['fence'].isnull()].index.tolist()
ames.loc[indexes, 'fence'] = 'NoFe'

In [422]:
# confirm changes
nulls_pct(ames, 'fence')
ames['fence'].value_counts()

Missing percentage: 0.0; Rows: 0


fence
NoFe     1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: count, dtype: int64

All rows successfully cleaned.

---
Column: `misc_feature`<br>
Nonimal data.

In [426]:
# check value counts
nulls_pct(ames, 'misc_feature')
ames['misc_feature'].value_counts(dropna = False)

Missing percentage: 96.83; Rows: 1986


misc_feature
NaN     1986
Shed      56
Gar2       4
Othr       3
TenC       1
Elev       1
Name: count, dtype: int64

Per data dictionary, this column can take on six different values.  They are: `Elev` for Elevator, `Gar2` for 2nd Garage..., `Othr` for Other, `Shed` for sheds over 100 SF, `TenC` for Tennis Court and `NA` for None.<br>
Change all `NaN` values to `NoFea` for no features instead of none.

In [431]:
# make the changes
indexes = ames[ames['misc_feature'].isnull()].index.tolist()
ames.loc[indexes, 'misc_feature'] = 'NoFea'

In [433]:
# confirm changes
nulls_pct(ames, 'misc_feature')
ames['misc_feature'].value_counts()

Missing percentage: 0.0; Rows: 0


misc_feature
NoFea    1986
Shed       56
Gar2        4
Othr        3
TenC        1
Elev        1
Name: count, dtype: int64